In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
#importing necessary libraries
#we'll use the make_regression function to generate a fake dataset for the purpose of learning regression

In [24]:
x, y, true_coefficients=make_regression(n_samples=1000, n_features=5, n_informative=5, coef=True)
#1000 samples are generated, with 5 features and all 5 will be relevant. True coefficients will also be returned

In [26]:
from sklearn.model_selection import train_test_split

In [28]:
x_train, x_test, y_train, y_test=train_test_split(x,y, random_state=5)

In [29]:
#now time to start building the Linear Regression object

In [58]:
class LR(): #abbreviate so as not to confuse with sklearn's LinearRegression
    
    def __init__(self, show_learning=False):
        self.show_learning=show_learning
        return
    
    def fit(self, predictors, target, learning_rate):
        
        '''predictors: x values
           target: y values
           learning_rate: size of step to take in gradient descent'''
        
        '''First we need to initialize the gradient and feature weights. For simplicity, just set them all to 1'''
        
        gradient=np.ones(len(predictors.T))
        weights=np.ones(len(predictors.T))
        
        count=0 #keep this count handy to see how quickly the machine learnsl
        
        '''We will be using the MSE (mean squared error) as the cost function. Calculus (and Andrew Ng) tells
        us that this function is convex meaning there is an absolute, global minimum whose slope is 0. We want
        to get as close to 0 as possible, so we can set a threshold and loop til we acvhieve that threshold'''
        
        while np.mean(abs(gradient)) >= 0.001:
            
            predictions=np.dot(predictors, weights)
            #this is equivalent to B1x1+B2x2+...Bnxn for each data point
            
            MSE=sum((predictions-target)**2) / 2*len(predictors)
            
            '''Basic steps for Gradient Descent
            
                1) Take the partial derivative with respect to each feature of the MSE equation
                2) Calculate the gradient for each sample and divide by total number of samples
                3) Update each feature as follows: weight=weight-learning_rate*avg_gradient
                4) Repeat until gradient is within acceptable (0.001) threshold'''
            
            '''partial_derivative(x1) = (prediction(x1)-y1)*(x1)
               avg_gradient(x1)=1/m*sum(predictions(x1)-y1)*x1)'''
            
            errors=predictions-target #array of all errors
            gradient=np.dot(predictors.T, errors)/len(predictors) #find the avg gradient for each featre
            weights -= gradient * learning_rate #simultaneously update the weights 
            count +=1
            
            if self.show_learning == True and count%20 == 0:
                
                print(f'MSE: {round(MSE,0)} | iteration: {count} | avg_gradient {round(np.mean(gradient),3)}')
                # every 20 iterations report on the learning 
                
        self.weights=weights
        self.count=count 
            
        return f"Fit complete: {count} iterations"
        
    def predict(self, test):

        return np.dot(test, self.weights)
        
    def score(self, predictors, target):

        SST=sum((target-np.mean(target)**2)) #Sum of Squared Total
        predictions=np.dot(predictors, self.weights)
        SSE=sum((target-predictions)**2) #Sum of Squared Error
        self.R2=1-(SSE/SST)

        return self.R2
        
    def coef(self):
        return self.weights


            
            
            
            
    

In [67]:
lr=LR(show_learning=True)

In [68]:
lr.fit(x_train,y_train, 0.1)

MSE: 98887793.0 | iteration: 20 | avg_gradient -7.299
MSE: 1227622.0 | iteration: 40 | avg_gradient -0.803
MSE: 15962.0 | iteration: 60 | avg_gradient -0.088
MSE: 216.0 | iteration: 80 | avg_gradient -0.01
MSE: 3.0 | iteration: 100 | avg_gradient -0.001


'Fit complete: 101 iterations'

In [69]:
lr.score(x_test, y_test)

1.0000000674709966

In [70]:
lr.coef()

array([23.45903378, 80.13829818,  6.50589233, 80.6175024 , 99.29270897])

In [71]:
true_coefficients

array([23.45896692, 80.13849981,  6.50611185, 80.61931119, 99.29463   ])

In [72]:
lr.coef()-true_coefficients

array([ 6.68603713e-05, -2.01626471e-04, -2.19523380e-04, -1.80878295e-03,
       -1.92103062e-03])

We were able to achieve a near perfect fit using the custom linear regression class. Now let's see how this compares to the sklearn LinearRegression()

In [74]:
from sklearn.linear_model import LinearRegression

In [75]:
sk_lr=LinearRegression()

In [76]:
sk_lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [77]:
sk_lr.score(x_test, y_test)

1.0

In [78]:
sk_lr.coef_

array([23.45896692, 80.13849981,  6.50611185, 80.61931119, 99.29463   ])

In [79]:
sk_lr.coef_-true_coefficients

array([-1.27897692e-13,  4.26325641e-14,  3.55271368e-15, -1.84741111e-13,
       -4.26325641e-14])

sklearn does a little bit better, but overall our custom function does a really great job!